In [1]:
!pip install pandas plotly

In [2]:
import pandas as pd

df0 = pd.read_csv('data/0_preprocessed.csv')
df0

,timestamp,duration,song,artist,album,song_artist
0,2019-07-05 03:00:26,2.45,7 rings,Ariana Grande,"thank u, next",7 rings - Ariana Grande
1,2019-07-05 03:01:44,1.29,All Falls Down,Alan Walker,All Falls Down,All Falls Down - Alan Walker
2,2019-07-05 03:05:23,3.67,I Don't Care,Ed Sheeran,I Don't Care,I Don't Care - Ed Sheeran
3,2019-07-05 03:08:41,3.30,Beautiful People,Ed Sheeran,Beautiful People,Beautiful People - Ed Sheeran
4,2019-07-05 03:12:01,3.32,All Falls Down,Alan Walker,All Falls Down,All Falls Down - Alan Walker
...,...,...,...,...,...,...
47672,2024-09-04 13:36:23,3.15,The Lazy Song,Bruno Mars,Doo-Wops & Hooligans,The Lazy Song - Bruno Mars
47673,2024-09-04 14:35:32,2.64,Part Of Me,Katy Perry,Teenage Dream: The Complete Confection,Part Of Me - Katy Perry
47674,2024-09-05 13:50:59,0.00,Beautiful In White,Shane Filan,Love Always,Beautiful In White - Shane Filan
47675,2024-09-05 16:08:47,0.07,Beautiful In White,Shane Filan,Love Always,Beautiful In White - Shane Filan


In [6]:
cur_song, cur_duration = '', 0
replayed = {} # key: song, value: [a,b,c] a 10+ b 30+ c 60+

for i, row in df0.iterrows():
  if cur_song != row['song']:
    a,b,c = replayed.get(cur_song, [0,0,0])
    if cur_duration < 30:
      pass
    else:
      if cur_duration < 45:
        a += 1
      elif cur_duration < 60:
        b += 1
      else:
        c += 1
      replayed[cur_song] = [a,b,c]
    cur_duration = 0

  cur_song = row['song']
  cur_duration += row['duration']

len(replayed)

37

In [39]:
df9 = pd.DataFrame(replayed, index=['30-45 mins','45-60 mins','60+ mins']).transpose()
df9 = df9.reset_index().melt(id_vars='index', var_name='duration', value_name='times')
df9 = df9.rename(columns={'index': 'song'})
scales = []
for t in df9['times']:
  if t == 0:
    t = 0
  elif 1 <= t <= 2:
    t = 5
  elif 3 <= t <= 5:
    t = 6
  elif 6 <= t <= 10:
    t = 7
  elif 11 <= t <= 15:
    t = 8
  else:
    t = 9

  scales.append(t)

df9['scales'] = scales
# df9.describe()
df9

,song,duration,times,scales
0,Delicate,30-45 mins,0,0
1,Everything Has Changed,30-45 mins,2,5
2,One Call Away,30-45 mins,2,5
3,Sparks Fly,30-45 mins,10,7
4,As Long As You Love Me,30-45 mins,3,6
...,...,...,...,...
106,Âm Thầm Bên Em,60+ mins,2,5
107,2AM,60+ mins,0,0
108,Suýt Nữa Thì...,60+ mins,6,7
109,Em Gái Mưa,60+ mins,1,5


In [40]:
range_r = []

In [54]:
import plotly.express as px

hover_data = {
    'scales': False,
    'times': True,
    'duration': True,
    'song': True
}

fig = px.bar_polar(df9,
                   r="scales",
                   theta="song",
                   color="duration",
                   template="plotly_dark",
                  #  range_r=[0,20],
                  #  log_r=True,
                   color_discrete_sequence= px.colors.sequential.Plasma_r,
                   hover_data=hover_data,
                  #  showticklabels=False
                   )

fig.update_layout(
    polar=dict(radialaxis=dict(showticklabels=False, ticks='')),
    showlegend=False)
fig.show()

In [ ]:
# prompt: do not include legends

fig.update_layout(showlegend=False)
fig.show()


In [53]:
df9.to_csv('9_wind_rose_replayer.csv', index=False)